In [2]:
import numpy as np
from scipy.io import loadmat


data = loadmat('ex3data1.mat')

print(data) 

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011', '__version__': '1.0', '__globals__': [], 'X': array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), 'y': array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]], dtype=uint8)}


In [4]:
print('X = \n',data['X']) 
print('y = \n ',data['y'])
print('X Shape = ' ,  data['X'].shape) 
print('Y Shape = ', data['y'].shape)

print(data['X'][0]) # first row
print('#####################################')
print(data['X'][0][155]) # first row and col 155(spasific info)

X = 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
y = 
  [[10]
 [10]
 [10]
 ...
 [ 9]
 [ 9]
 [ 9]]
X Shape =  (5000, 400)
Y Shape =  (5000, 1)
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+

In [6]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cost(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg


def gradient_with_loop(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    
    error = sigmoid(X * theta.T) - y
    
    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        
        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((learningRate / len(X)) * theta[:,i])
    
    return grad
def gradient(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y
    
    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)
    
    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)
    
    return np.array(grad).ravel()


In [8]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0] #5000
    params = X.shape[1] #400
    
    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))
    
    print('all_theta shape ' , all_theta.shape)
    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    print('X shape ' , X.shape)
  # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))
        
        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x
    
    return all_theta


In [9]:
rows = data['X'].shape[0]
params = data['X'].shape[1]

print('rows = ' ,rows)
print('params = ' , params)

print('===================================================')


all_theta = np.zeros((10, params + 1))

print('all_theta \n' , all_theta)
print('all_theta shape \n' , all_theta.shape)

rows =  5000
params =  400
all_theta 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
all_theta shape 
 (10, 401)


In [10]:
X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)


print(X) 
print('X Shape = ' ,  X.shape) 

print('===================================================')

theta = np.zeros(params + 1)

print('theta \n' , theta ) 

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]
X Shape =  (5000, 401)
theta 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 

In [11]:
y_0 = np.array([1 if label == 0 else 0 for label in data['y']])

print('y_0')
print(y_0.shape)
print(y_0)

y_0
(5000,)
[0 0 0 ... 0 0 0]


In [12]:
y_0 = np.reshape(y_0, (rows, 1))


print('y_0')
print(y_0.shape)
print(y_0)

y_0
(5000, 1)
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]


In [14]:
print('X.shape = ',X.shape)
print()
print('y.shape = ',y_0.shape)
print()
print('theta.shape = ',theta.shape)
print()
print('all_theta.shape = ',all_theta.shape)
print()
print('data array = ' , np.unique(data['y']))


X.shape =  (5000, 401)

y.shape =  (5000, 1)

theta.shape =  (401,)

all_theta.shape =  (10, 401)

data array =  [ 1  2  3  4  5  6  7  8  9 10]


In [15]:
all_theta = one_vs_all(data['X'], data['y'], 1, 1) # 10% from thetas

print('Theta shape =   ' , all_theta.shape)
print('Theta = ')
print(all_theta)

all_theta shape  (1, 401)
X shape  (5000, 401)
Theta shape =    (1, 401)
Theta = 
[[-3.70247929e-05  0.00000000e+00  0.00000000e+00  1.45740956e-10
  -1.26000448e-09 -1.07067739e-09 -2.29267665e-08 -2.08810304e-08
  -2.32625492e-11 -1.77197006e-08 -1.42717371e-08 -3.33472945e-09
   2.45288428e-09  4.29598311e-09  6.83398815e-09  9.29087983e-09
   5.08966530e-09  1.40383167e-09  4.28809830e-11 -1.79446410e-11
   0.00000000e+00 -1.99645439e-12  2.16948408e-11 -1.85206015e-09
   1.74264636e-08  2.23711398e-08 -5.02669571e-08 -1.41936269e-07
  -2.11236025e-07 -2.07880613e-07 -1.09764260e-07 -8.71206087e-09
   1.24087888e-08 -3.78652039e-09 -1.69654972e-08 -4.16799827e-08
  -1.74641879e-08  1.74915640e-09  6.56556695e-09  1.60534528e-09
   5.57721637e-10  1.48933216e-11 -3.36720615e-10  6.07719575e-10
   2.75179856e-08 -4.77049313e-08 -5.16705708e-07 -9.73763764e-07
  -1.19167516e-06 -9.55309560e-07 -5.09174896e-07 -3.56951543e-07
  -2.88959052e-07 -3.74443704e-07 -7.49724899e-07 -8.7293469

In [16]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)
    
    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)
    
    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1
    
    return h_argmax


In [18]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))
print("low accuracy becouse of only 10% from thetas")

accuracy = 10.0%
low accuracy becouse of only 10% from thetas


In [19]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1) # all thetas


all_theta shape  (10, 401)
X shape  (5000, 401)


In [20]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 74.6%
